## Light Gradient Boosting Machine - modelo de comparação

A biblioteca LightGBM é uma estrutura de aumento de gradiente que usa algoritmos de aprendizagem baseados em árvore para implementar este modelo. Ele é distribuível e capaz de lidar com grandes volumes de dados com alto desempenho. LightGBM é ideal para uma variedade de tarefas de aprendizado de máquina, como classificação, regressão e ranqueamento, graças à sua velocidade e eficiência.

### Instruções para uso local ou remoto (Google Colab ou VScode)

Aqui deixaremos brevemente um passo a passo para que você usuário seja capaz de executar o código localmente ou remotamente pelo seu google drive, podendo escolher a forma mais viável para seu uso e conhecimento.

##### Google Colab

1. Faça o upload do seu arquivo .ipynb para o Google Drive.
2. Abra o Google Colab em seu navegador.
3. Clique em "Arquivo" no menu superior e selecione "Abrir notebook".
4. Na guia "Upload", clique em "Procurar" e selecione o arquivo .ipynb que você enviou para o Google Drive.
5. Após selecionar o arquivo, clique em "Abrir".
6. Aguarde o carregamento do notebook no Google Colab.
7. Agora que você carregou o notebook no Google Colab, você pode fazer as alterações necessárias nos arquivos e caminhos para se adequar ao seu ambiente específico.

No notebook, a célula seguinte à essa contém as leituras dos arquivos CSV com o caminho do drive do criador desse notebook.
Comente as linhas que fazem referência aos arquivos locais e descomente as linhas que fazem referência ao Google Drive. Por exemplo:

- descomente as linhas que começam com # from google.colab import drive;
- comente as linhas que começam com tabela_Meta = pd.read_csv("./data/Cópia de BASE INTELI_META_OCUP-limpo.csv").

Certifique-se de que os arquivos CSV estejam localizados no diretório correto em seu ambiente virtual. Por exemplo, se você tiver uma pasta chamada "data" no mesmo diretório do notebook, coloque os arquivos CSV nessa pasta e ajuste seus nomes. Possivelmente os arquivos vão seguir o seguinte padrão, mesmo no seu drive:

tabela_Meta = pd.read_csv("/content/drive/MyDrive/NomeDaPastaDosArquivos/arquivo.csv)

Salve as alterações no notebook.
Agora você pode executar as células do notebook no Google Colab e as alterações nos arquivos e caminhos serão aplicadas ao seu ambiente do colab. Lembre-se de que você não precisará ter as bibliotecas necessárias instaladas em seu ambiente local para executar o código corretamente, dado que ao executar pela ferramenta do google essas dependências estarão aplicadas por padrão

##### VScode

1. Coloque os arquivos CSV dentro da pasta "data" desse notebook

No notebook, a célula seguinte à essa contém as leituras dos arquivos CSV com o caminho do drive do criador desse notebook.
Comente as linhas que fazem referência aos arquivos locais e descomente as linhas que fazem referência ao Google Drive. Por exemplo:

- descomente as linhas que começam com # from google.colab import drive;
- comente as linhas que começam com tabela_Meta = pd.read_csv("./data/Cópia de BASE INTELI_META_OCUP-limpo.csv").

Certifique-se de que os arquivos CSV estejam localizados no diretório correto em seu ambiente virtual. Por exemplo, se você tiver uma pasta chamada "data" no mesmo diretório do notebook, coloque os arquivos CSV nessa pasta e ajuste seus nomes. Possivelmente os arquivos vão seguir o seguinte padrão, mesmo no seu drive:

tabela_Meta = pd.read_csv("./data/Cópia de BASE INTELI_META_OCUP-limpo.csv")

Salve as alterações no notebook.
Agora você pode executar as células do notebook no VScode e as alterações nos arquivos e caminhos serão aplicadas ao seu ambiente do colab. Lembre-se de que você precisará ter as bibliotecas necessárias instaladas em seu ambiente local para executar o código corretamente:

Para fazer a instalação, basta abrir o terminal integrado e inserir o seguinte:

In [1]:
#pip install lightgbm pandas numpy scikit-learn matplotlib shap

Feito isso vamos importar as bibliotecas e dependências usadas para nosso notebook.

In [ ]:
try:
    import lightgbm as lgb # type: ignore
    import pandas as pd # type: ignore
    import matplotlib.pyplot as plt # type: ignore
    from lightgbm import LGBMRegressor # type: ignore
    from sklearn.model_selection import RandomizedSearchCV # type: ignore
    import numpy as np # type: ignore
    from sklearn.preprocessing import LabelEncoder # type: ignore
    from sklearn.model_selection import train_test_split # type: ignore
    from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score # type: ignore
    from math import sqrt
    import shap
    print("Todos os pacotes foram carregados com sucesso!")
except ModuleNotFoundError as e:
    print(f"Erro ao carregar pacotes: {e}")

Agora iremos importar as tabelas já limpas.

In [3]:
# # pegando arquivos csv do drive
# from google.colab import drive
# drive.mount('/content/drive')

# # Lendo os arquivos CSV
# tabela_Meta = pd.read_csv("/content/drive/MyDrive/Primeiro Ano/Módulo 3 - Modelo Preditivo Gazeta/Base de Dados Limpas/BASE INTELI_META-limpo.csv")
# tabela_Agosto = pd.read_csv("/content/drive/MyDrive/Primeiro Ano/Módulo 3 - Modelo Preditivo Gazeta/Base de Dados Limpas/tratada_BaseDados_ProjetoINTELI_RG_01_AGOSTO_2024.csv")

# para realizar o processo localmente, descomente as linhas abaixo e comente as linhas acima.
tabela_Meta = pd.read_csv("../data/dados_tratados/BASE INTELI_META-limpo.csv")
tabela_Agosto = pd.read_csv("../data/dados_tratados/tratada_BaseDados_ProjetoINTELI_RG_01_AGOSTO_2024.csv")

Com os arquivos em mão, vamos definir valores máximos no formato de exibição e agrupar colunas, visando somar as de valor númerico de acordo com as exigências do parceiro. No fim, a saída mostrará uma representação da tabela filtrada com as features usadas será mostrada.

In [ ]:
# Definindo formato de exibição
pd.set_option('display.float_format', '{:.0f}'.format)

# Agrupar e somar 'Vl Liquido Final' e 'Outra Coluna'
tabela_agosto_segmento = tabela_Agosto.groupby(['Ano', 'Mês', 'Veiculo', 'Origem', 'Segmento']).agg({
    'Vl Liquido Final': 'sum',
    # 'VL Tabela': 'sum'

}).reset_index()

# tabela_agosto_segmento['VL Tabela'] = tabela_agosto_segmento['VL Tabela'] / 1000
tabela_agosto_segmento.head()

 Abaixo, as colunas categóricas (aquelas que contém texto) serão transformadas em numéricas, de modo a pouco comprometer o desempenho do modelo.

In [5]:
categorical_columns = ['Veiculo', 'Origem', 'Segmento']
X = tabela_agosto_segmento.drop('Vl Liquido Final', axis=1)
y = tabela_agosto_segmento['Vl Liquido Final']

label_encoders = {}
for col in categorical_columns:
    le = LabelEncoder()
    X[col] = le.fit_transform(X[col])
    label_encoders[col] = le


Após, faremos a divisão das amostras de treinamento e teste do modelo, e depois o ajuste do modelo com o grupo de treino, equivalente à 30% da base total

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=4)

modelo = lgb.LGBMRegressor()
modelo.fit(X_train, y_train)

#### Grid Search

Usando grid search, podemos encontrar os hyperparâmetros ideais para o modelo de machine learning. Funciona por meio de uma busca abrangente em um subconjunto específico do espaço hiperparamétrico.

Em resumo, essa técnica permite que testemos diversos modelos com diferentes parâmetros de entrada de forma automática, ao criar intervalos que desejamos que o modelo seja testado.

Nota: devido ao longo tempo de execução, o código responsável está comentado, mas seu output será usado nos blocos de código posteriores!

In [7]:
# param_grid = {
#     'n_estimators': list(range(40, 500)),
#     'learning_rate': [0.01, 0.1, 0.2],
#     'max_depth': [3, 4, 5, 6, 7, 9 , 11, 13, 15],
#     'subsample': [0.1, 0.3, 0.4, 0.7, 1],
#     'min_child_weight': [0.5, 1, 2, 3, 4, 5],
#     'colsample_bytree': [0.7, 0.8, 1], 
#     'reg_alpha': [0, 0.01, 0.1],        
#     'reg_lambda': [1, 1.5, 2]           
# }

# # Configurando o Grid Search com validação cruzada
# grid_search = RandomizedSearchCV(
#     estimator=LGBMRegressor(),
#     param_distributions=param_grid,
#     scoring='neg_mean_absolute_error',
#     cv=10,
#     verbose=0,
#     n_jobs=8,
#     n_iter=500
# )

# # Ajustando o Grid Search ao conjunto de dados
# grid_search.fit(X_train, y_train)

# # Imprimindo os melhores parâmetros e o melhor resultado
# print("Melhores Parâmetros:", grid_search.best_params_)
# print("Melhor MAE:", -grid_search.best_score_)

Agora que temos os melhores parâmetros, ajustaremos o modelo conforme aos melhores resultados:

In [ ]:
modelo = lgb.LGBMRegressor(subsample=0.1, reg_lambda=1, reg_alpha=0.1, n_estimators=493, min_child_weight=5, max_depth=13, learning_rate=0.1, colsample_bytree=1)
modelo.fit(X_train, y_train)

Por fim, faremos o modelo realizar previsões com a porcentagem de 70% da base que foi destinada a esse teste.

Podemos ver na saída desse bloco de código as métricas usadas para validar a eficiência do modelo, explicaremos elas em breve.

In [ ]:
y_pred = modelo.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
mape = np.mean(np.abs((y_test - y_pred) / y_test)) / 10

print(f"Mean Absolute Error: {mae}")
print(f"Root Mean Squared Error: {sqrt(rmse)}")
print(f"R² Score: {r2}")
print(f'MPAE: {mape:.2f}')

#### Erro Médio Absoluto (MAE - do inglês *Mean Absoluto Error*)

O MAE é uma métrica utilizada para avaliar a qualidade de modelos de regressão, calculando a média das diferenças absolutas entre os valores reais e os valores preditos. Quanto menor o valor do MAE, melhor o desempenho do modelo. Abaixo, apresentamos o cálculo desse indicador no modelo do grupo:<br>

In [ ]:
mae = mean_absolute_error(y_test, y_pred)
print(f"Mean Absolute Error: {mae}")

#### Raiz do Erro Quadrático Médio (RMSE - do inglês *Root Mean Squared Error*)

O RMSE (Root Mean Squared Error) é uma métrica de avaliação utilizada em modelos de regressão que, assim como o MSE (Mean Squared Error), mede a diferença entre os valores reais e os valores previstos. No RMSE, essa diferença é elevada ao quadrado, o que elimina os sinais negativos e dá mais peso a grandes erros. A diferença principal está no fato de que, ao final, é aplicada a raiz quadrada no valor resultante, tornando a métrica mais interpretável, pois os erros são trazidos de volta à mesma escala das variáveis originais. Portanto, quanto menor o RMSE, melhor o desempenho do modelo.

In [ ]:
rmse = mean_squared_error(y_test, y_pred)
print(f"Root Mean Squared Error: {sqrt(rmse)}")

#### Coeficiente de Determinação (R²)

O Coeficiente de Determinação R² é uma métrica que representa o percentual da variância dos dados previstos, ou seja, o quão explicativo é o modelo em relação aos dados de acordo com o quão distante esses valores estão do valor central (médio). Uma vez que a fórmula do R² considera a subtração desta conta por 1, quando menor o percentual obtido, melhor é a explicação do modelo. Este, no entanto, não é suficiente para ter uma noção geral da performace do modelo, dependendo de outras métricas (como o MAE e o MSE). Abaixo, apresentamos o cálculo desse indicador no modelo do grupo:

In [ ]:
r2 = r2_score(y_test, y_pred)
print(f"R² Score: {r2}")

#### Erro Percentual Médio Absoluto (MAPE)

O Erro Percentual Médio Absoluto (MAPE) mede a precisão de modelos de regressão, calculando a média dos erros percentuais absolutos entre os valores previstos e reais. Ao expressar o erro como uma proporção entre 0 e 1, o MAPE facilita a interpretação do desempenho relativo do modelo. Quanto mais próximo de 0, melhor a precisão; valores próximos de 1 indicam maiores erros. Embora seja útil para avaliar a precisão relativa, o MPAE deve ser complementado por outras métricas para uma análise mais completa. 

In [ ]:
mape = np.mean(np.abs((y_test - y_pred) / y_test)) / 10
print(f'MPAE: {mape:.2f}')

A seguir, vamos analisar os resultados visualmente:

O gráfico compara os valores reais com as previsões do modelo. A linha roxa representa as previsões geradas pelo modelo, enquanto os pontos vermelhos correspondem aos valores reais. Quanto mais próximos os pontos estiverem da linha, melhor a qualidade das previsões.

In [ ]:
plt.figure(figsize=(10, 6))
plt.scatter(y_test, y_pred, color='red', alpha=0.6)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], color='purple', lw=3)
plt.xlabel('Valores Reais')
plt.ylabel('Previsões')
plt.title('Comparação entre Valores Reais e Previsões - Modelo LightGBM')
plt.grid(True)
plt.show()

### Conlusões

Apesar de obter resultados factualmente positivos, é nítido que ao compararmos com o modelo entregue na sprint 3 de desenvolvimento, ainda existe uma desvantagem entre o resultado do LightGBM em relação ao Random Forest Regressor, como observado à seguir:

Métrica | Random Forest Regressor | Light GBM Regressor
--- | --- | ---
R² | 91% | 90%
MAE | 10.417 | 13.329
RMSE | 31.757 | 33.536
MAPE | 0,40 | 0,77

O R² do LightGBM é apenas 1% menor, indicando que ambos os modelos explicam bem a variabilidade dos dados. No entanto, o MAE e o RMSE mostram que o Random Forest possui menor erro absoluto e quadrático, respectivamente, sugerindo uma performance mais precisa. O MAPE reforça essa tendência, com o erro percentual absoluto do LightGBM sendo quase o dobro do Random Forest (0,77 contra 0,40), apontando para uma menor precisão nas previsões do LightGBM na comparação.

## Explicabilidade
A explicabilidade de um modelo preditivo refere-se à capacidade de compreender e interpretar como um modelo toma suas decisões ou faz previsões. Isso é particularmente importante em modelos complexos onde o processo de decisão não é facilmente visível para os seres humanos. Logo abaixo, vamos iniciar a explicabilidade do modelo para conferir como as previsões foram realizadas.

In [ ]:
# Crie um objeto explainer SHAP
explainer = shap.Explainer(modelo, X_train)

# Calcule os valores SHAP para as previsões
shap_values = explainer(X_test)


#### Summary Plot
Esse gráfico de resumo mostra a importância global das variáveis no modelo, bem como a distribuição dos valores SHAP (impacto no resultado do modelo) para cada variável. Ele mostra como cada variável impacta globalmente o modelo e sua distribuição nos dados.

In [ ]:
shap.summary_plot(shap_values, X_test)

- Eixo Y: Lista as variáveis/features do modelo, como "Segmento", "Veículo", "Origem", etc.
- Eixo X: Representa os valores SHAP, que indicam o impacto da variável no resultado do modelo. Valores positivos aumentam a previsão, enquanto valores negativos a reduzem.
- Pontos: Cada ponto representa uma observação (amostra) dos seus dados.
A cor do ponto indica o valor da variável: azul para valores baixos, vermelho para valores altos.
A posição horizontal do ponto indica o impacto que aquela observação teve no resultado. Quanto mais à direita, maior o impacto positivo na previsão; quanto mais à esquerda, maior o impacto negativo.

A variável Segmento tem um impacto maior em geral, com muitos pontos azuis e vermelhos espalhados. Isso indica que, dependendo do valor da variável (seja alto ou baixo), ela pode ter um impacto positivo ou negativo nas previsões.
Origem também tem uma distribuição de valores SHAP, mas tem menos impacto em comparação com "Segmento" e "Veículo".

#### Waterfall Plot
Este gráfico explica a previsão individual de um dado ponto de teste. Ele decompõe o valor predito em contribuições individuais de cada variável. O valor final predito é a soma da base do modelo com as contribuições de cada variável. Ou seja, ele explica como cada variável contribui para a previsão de um ponto específico, mostrando o impacto individual de cada uma na previsão final.

In [ ]:

# Mostra como cada recurso impactou uma previsão específica.
shap.waterfall_plot(shap_values[0])

- f(x): O valor predito para essa observação específica (no caso, aproximadamente -47675.89).
- Base value: O valor de referência do modelo, antes de considerar os efeitos das variáveis (aproximadamente 47850.108).
- Barras Azuis/Vermelhas:
Barras azuis indicam que a variável diminuiu a previsão em relação ao valor base.
Barras vermelhas indicam que a variável aumentou a previsão.
Cada barra mostra o valor da variável e seu impacto específico na previsão.

A variável Segmento (com valor 7) reduziu a previsão em -36360.5.
A variável Veículo (com valor 13) também teve um impacto negativo, reduzindo a previsão em -39085.32.
Por outro lado, a variável origem aumentou a precisão em 2799.62.